In [5]:
%pip install xgboost lightgbm catboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   -- ------------------------------------- 4.5/72.0 MB 24.5 MB/s eta 0:00:03
   ----------- ---------------------------- 21.2/72.0 MB 54.8 MB/s eta 0:00:01
   ---------------------- ----------------- 41.2/72.0 MB 68.7 MB/s eta 0:00:01
   --------------------------------- ------ 59.8/72.0 MB 73.5 MB/s eta 0:00:01
   ---------------------------------------  71.8/72.0 MB 76.0 MB/s eta 0:00:01
   ---------------------------------------- 72.0/72.0 MB 67.9 MB/s  0:00:01
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 46.2 MB/s  0:00:00
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ------- -------------------------------- 18.9/102.4 MB 91.6 MB/s eta 0:00:01
   -------------- ------------------------- 37.7/102.4 MB 90.9 MB/s eta 0:00:01
   --------------------- ------------------ 55.1/102.4 MB 87.8 MB/s eta 0:00:01
   ---

In [1]:
import pandas as pd
import numpy as np
import gc
import os

# ==========================================
# 1. UTILITY: MEMORY REDUCER (The "Quant" Trick)
# ==========================================
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32) # float16 is unstable, use 32
                else:
                    df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# ==========================================
# 2. LOAD & MERGE
# ==========================================
data_path = '../data/' 

print("Loading Train Targets...")
train_df = pd.read_csv(os.path.join(data_path, 'train_2016_v2.csv'))

print("Loading Properties (Features)...")
# This file is large, so we process it carefully
prop_df = pd.read_csv(os.path.join(data_path, 'properties_2016.csv'))

print("Optimizing Memory...")
prop_df = reduce_mem_usage(prop_df)

print("Merging Data...")
# We only care about homes that were actually sold (train_df)
# Left Join: Keep all sold homes, attach their properties
df = pd.merge(train_df, prop_df, on='parcelid', how='left')

# CLEANUP: Delete the heavy raw frames to free RAM
del train_df, prop_df 
gc.collect()

print(f"✅ Dataset Ready. Shape: {df.shape}")
print(df.head())

Loading Train Targets...
Loading Properties (Features)...


C:\Users\Admin49\AppData\Local\Temp\ipykernel_20280\231967511.py:53: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  prop_df = pd.read_csv(os.path.join(data_path, 'properties_2016.csv'))


Optimizing Memory...
Memory usage of dataframe is 1320.97 MB
Memory usage after optimization is: 717.43 MB
Decreased by 45.7%
Merging Data...
✅ Dataset Ready. Shape: (90275, 60)
   parcelid  logerror transactiondate  airconditioningtypeid  \
0  11016594    0.0276      2016-01-01                    1.0   
1  14366692   -0.1684      2016-01-01                    NaN   
2  12098116   -0.0040      2016-01-01                    1.0   
3  12643413    0.0218      2016-01-02                    1.0   
4  14432541   -0.0050      2016-01-02                    NaN   

   architecturalstyletypeid  basementsqft  bathroomcnt  bedroomcnt  \
0                       NaN           NaN          2.0         3.0   
1                       NaN           NaN          3.5         4.0   
2                       NaN           NaN          3.0         2.0   
3                       NaN           NaN          2.0         2.0   
4                       NaN           NaN          2.5         4.0   

   buildingclass

In [2]:
import plotly.express as px

# ==========================================
# 3. MISSING VALUE ANALYSIS
# ==========================================
# Calculate missing percentage per column
missing = df.isnull().sum() / len(df)
missing = missing[missing > 0].sort_values(ascending=False)

print(f"Columns with missing data: {len(missing)}")

# Visualize the "Sparseness" (Great for the report)
fig = px.bar(x=missing.index[:20], y=missing.values[:20], 
             labels={'x':'Feature', 'y':'Percent Missing'},
             title='Top 20 Missing Features (The "Dirty Data" Challenge)')
fig.show()

# ==========================================
# 4. DOMAIN-SPECIFIC IMPUTATION
# ==========================================
print("\nApplying Smart Imputation...")

# A. The "Assumption of Zero" (NaN implies feature doesn't exist)
# If 'poolcnt' is missing, the house likely has no pool.
zero_fill_cols = [
    'poolcnt', 'pooltypeid2', 'pooltypeid7', 'pooltypeid10', 'hashottuborspa', 
    'decktypeid', 'finishedsquarefeet6', 'finishedsquarefeet12', 'finishedsquarefeet13', 
    'finishedsquarefeet15', 'finishedsquarefeet50', 'fireplacecnt', 
    'garagecarcnt', 'garagetotalsqft', 'threequarterbathnbr', 'numberofstories'
]

for col in zero_fill_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# B. The "Assumption of Average" (Must have a value)
# If 'taxamount' is missing, we fill with the median (robust to outliers).
df['taxvaluedollarcnt'] = df['taxvaluedollarcnt'].fillna(df['taxvaluedollarcnt'].median())
df['structuretaxvaluedollarcnt'] = df['structuretaxvaluedollarcnt'].fillna(df['structuretaxvaluedollarcnt'].median())
df['landtaxvaluedollarcnt'] = df['landtaxvaluedollarcnt'].fillna(df['landtaxvaluedollarcnt'].median())
df['taxamount'] = df['taxamount'].fillna(df['taxamount'].median())

# C. Mode Imputation (Categorical / Location)
# If zip code is missing, use the most common one.
df['regionidzip'] = df['regionidzip'].fillna(df['regionidzip'].mode()[0])
df['yearbuilt'] = df['yearbuilt'].fillna(df['yearbuilt'].median())

# D. Drop columns that are still >90% empty (Too risky to impute)
remaining_missing = df.isnull().sum() / len(df)
drop_cols = remaining_missing[remaining_missing > 0.90].index.tolist()
print(f"Dropping {len(drop_cols)} columns with >90% missing data: {drop_cols}")
df = df.drop(columns=drop_cols)

print(f"✅ Cleanup Complete. New Shape: {df.shape}")

Columns with missing data: 47



Applying Smart Imputation...
Dropping 12 columns with >90% missing data: ['architecturalstyletypeid', 'basementsqft', 'buildingclasstypeid', 'finishedfloor1squarefeet', 'poolsizesum', 'storytypeid', 'typeconstructiontypeid', 'yardbuildingsqft17', 'yardbuildingsqft26', 'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear']
✅ Cleanup Complete. New Shape: (90275, 48)


In [3]:
# ==========================================
# 5. FEATURE ENGINEERING (The Alpha)
# ==========================================
print("Creating Interaction Features...")

# A. Value Density (Price per Square Foot)
# This is the #1 metric in real estate.
# We use 'structuretaxvaluedollarcnt' (Building Value) / 'calculatedfinishedsquarefeet' (Size)
df['structure_dollar_per_sqft'] = df['structuretaxvaluedollarcnt'] / df['calculatedfinishedsquarefeet']
df['land_dollar_per_sqft'] = df['landtaxvaluedollarcnt'] / df['lotsizesquarefeet']

# B. Room Ratios (Quality of Life)
# A 4-bedroom house with 1 bathroom is worth LESS than a 3-bed/2-bath.
# We add 0.001 to avoid DivisionByZero errors.
df['bed_bath_ratio'] = df['bedroomcnt'] / (df['bathroomcnt'] + 0.001)

# C. Temporal Features
# How old is the house? (Newer homes usually have lower maintenance risk)
df['age'] = 2016 - df['yearbuilt']

# D. Tax Efficiency (Effective Tax Rate)
# Some areas have higher tax rates, which lowers the property's investment appeal.
df['tax_rate'] = df['taxamount'] / df['taxvaluedollarcnt']

# E. Neighborhood Context (The "Relative" Value)
# "Is this house bigger than its neighbors?"
# We group by Zip Code (regionidzip) and calculate the average size.
zip_stats = df.groupby('regionidzip')['calculatedfinishedsquarefeet'].mean().to_dict()
df['avg_sqft_in_zip'] = df['regionidzip'].map(zip_stats)

# The Signal: Ratio of House Size to Neighborhood Avg
df['sqft_vs_zip_avg'] = df['calculatedfinishedsquarefeet'] / df['avg_sqft_in_zip']

# --- Cleanup ---
# Division often creates Infinite values (div by 0). We treat them as NaNs, then fill with 0.
df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

print(f"✅ Feature Engineering Complete. New Shape: {df.shape}")

# --- Sanity Check ---
# Let's see which of our new features correlates most with the Error (logerror)
new_features = ['structure_dollar_per_sqft', 'land_dollar_per_sqft', 'bed_bath_ratio', 'age', 'tax_rate', 'sqft_vs_zip_avg']
print("\nCorrelation with Target (Log Error):")
print(df[new_features + ['logerror']].corr()['logerror'].sort_values(ascending=False))

Creating Interaction Features...
✅ Feature Engineering Complete. New Shape: (90275, 55)

Correlation with Target (Log Error):
logerror                     1.000000
sqft_vs_zip_avg              0.040677
bed_bath_ratio               0.012588
structure_dollar_per_sqft   -0.000416
tax_rate                    -0.003674
land_dollar_per_sqft        -0.012707
age                         -0.017075
Name: logerror, dtype: float64


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import xgboost as xgb

# ==========================================
# 6. FINAL DATA PREP
# ==========================================
print("Preparing Training Data...")

# Drop ID and Target columns
drop_cols = ['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode']
# Remove columns that are not in the dataframe (safety check)
drop_cols = [c for c in drop_cols if c in df.columns]

X = df.drop(columns=drop_cols)
y = df['logerror']

# HANDLE CATEGORICALS: Tree models need numbers, not strings.
# We Label Encode any remaining "object" (string) columns.
for col in X.columns:
    if X[col].dtype == 'object':
        lbl = LabelEncoder()
        # Convert to string to handle mixed types/NaNs safely for encoding
        X[col] = lbl.fit_transform(X[col].astype(str))

# Split: 80% Train, 20% Validate
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training on {X_train.shape[0]} rows, Validating on {X_val.shape[0]} rows.")

# ==========================================
# 7. DEFINING THE STACK (The "Brain")
# ==========================================
# Base Models (Level 0)
estimators = [
    ('lgbm', lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05, random_state=42)),
    ('xgb', xgb.XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=42, n_jobs=-1))
]

# Meta Learner (Level 1)
# Linear Regression finds the optimal weighted average of the base models.
stack = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression(),
    cv=3 # 3-Fold Cross Validation inside the stack
)

# ==========================================
# 8. TRAINING & EVALUATION
# ==========================================
print("\n🚀 Training Stacking Ensemble (this may take 1-2 mins)...")
stack.fit(X_train, y_train)

# Predict
y_pred = stack.predict(X_val)

# Metric: Mean Absolute Error (MAE) - The Official Zillow Metric
mae = mean_absolute_error(y_val, y_pred)
print(f"\n✅ Final Validation MAE: {mae:.6f}")

# Benchmark: What if we just predicted the MEAN every time?
baseline_mae = mean_absolute_error(y_val, [y_train.mean()] * len(y_val))
print(f"   Baseline (Mean) MAE: {baseline_mae:.6f}")
print(f"   Improvement: {100 * (baseline_mae - mae) / baseline_mae:.2f}%")

Preparing Training Data...
Training on 72220 rows, Validating on 18055 rows.

🚀 Training Stacking Ensemble (this may take 1-2 mins)...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014888 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6119
[LightGBM] [Info] Number of data points in the train set: 72220, number of used features: 49
[LightGBM] [Info] Start training from score 0.011527
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009971 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6060
[LightGBM] [Info] Number of data points in the train set: 48146, number of used features: 49
[LightGBM] [Info] Start training from score 0.011722
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] T

In [5]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# ==========================================
# 9. EXPLAINABILITY (Opening the Black Box)
# ==========================================

# We extract the first model from our stack (LightGBM)
# Access path: Stack -> Estimators_ (list) -> LightGBM -> feature_importances_
lgbm_model = stack.estimators_[0] 

# Create a DataFrame of Feature Importances
feature_imp = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': lgbm_model.feature_importances_
})

# Sort by Importance
feature_imp = feature_imp.sort_values(by='Importance', ascending=False).head(15)

# Plot
fig = px.bar(feature_imp, x='Importance', y='Feature', orientation='h',
             title='Global Feature Importance (What drives the Zestimate Error?)',
             color='Importance', color_continuous_scale='Viridis')

# Invert y-axis so top feature is at the top
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

print("Top 3 Drivers of Valuation Error:")
print(feature_imp.head(3)['Feature'].values)

Top 3 Drivers of Valuation Error:
['lotsizesquarefeet' 'taxamount' 'sqft_vs_zip_avg']


In [6]:
import joblib
joblib.dump(stack, 'zillow_pricing_engine.pkl')
print("✅ Model saved to zillow_pricing_engine.pkl")

✅ Model saved to zillow_pricing_engine.pkl
